<a href="https://colab.research.google.com/github/poudel-bibek/Intro-to-AI-Assignments/blob/main/A5_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![inh](https://user-images.githubusercontent.com/96804013/152455353-b69e07d7-d856-4b1a-abd4-bb1c85a22a7d.png)


# Assignment 5: Long-Short Term Memory (Task)
---

In this assignment, we will use Long-Short Term Memory (LSTM) ([link](https://en.wikipedia.org/wiki/Long_short-term_memory)) to predict the number of passengers in an airline for the next day given the number of passengers today. The data is represented as a time series.

First, let's import necessary libraries.

                    
                    import math
                    import numpy as np
                    import pandas as pd
                    import matplotlib.pyplot as plt
                    from sklearn.preprocessing import MinMaxScaler
                    from sklearn.metrics import mean_squared_error
                    from sklearn.model_selection import train_test_split

                    from tensorflow import keras
                    from keras.models import Sequential
                    from keras.layers import Dense, LSTM


The code for downloading the data and visualizing a subset of it is provided below.

                    airlines = pd.read_csv('https://github.com/poudel-bibek/Intro-to-AI-Assignments/files/7999363/airline-passengers.csv')
                    print(f"Dataset\n{airlines.head()}\n")

                    subset_size = 40 
                    subset_months= airlines['Month'][0:subset_size]
                    subset_passengers = airlines['Passengers'][0:subset_size]

                    fig, ax = plt.subplots(figsize=(14,4), dpi = 80)
                    ax.plot(range(len(subset_months)), np.array(subset_passengers), '+', label = "Passenger count")
                    ax.plot(range(len(subset_months)), np.array(subset_passengers), alpha = 0.5)
                    ax.set_xticks(range(len(subset_months))) 
                    ax.set_xticklabels(subset_months); # ';' prevents displaying auxillary outputs
                    plt.xticks(rotation=45);
                    ax.legend()


<p align="center">
  <img src="https://user-images.githubusercontent.com/96804013/153318289-1be2e715-5180-48f7-bdb1-9f41b9952b4c.png")
"/>
</p>

<p align="center">
  <em>Figure 1:Visualization of raw dataset</em>
</p>




We will only be working with the Passengers column. So the dates column can be discarded. 

Use the code below to:
- extract Passengers column 
- take a peek into it 
- convert its type 
- rehsape it 
- print its shape along with minimum and maximum values in it 

                    passengers = airlines['Passengers']
                    print(f"Passengers column\n{passengers.head()}")

                    dataset = passengers.values.astype('float32').reshape(-1,1)
                    print(f"\nBefore Normalization :\n shape ={dataset.shape}, max = {np.max(dataset)}, min = {np.min(dataset)}")

---
## Exercise 1: 

Normalize the data i.e., the `dataset` variable, using Minmax Scaler ([link](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)): 
  - Instantiate a MinMaxScaler() object and make the feature range between 0 and 1
  - Use the fit_transform function of MinMaxScaler to scale the dataset
  - Print the shape of dataset, max and min values after normalization. 
  
The shape of `(144,1)` means the data looks like this `[[data 0], [data 1], [data 2].... [data 143]]`

Now use the following code to reshape  the dataset to have a shape of `(144,)` and print the shape of the dataset, now we have made the data look like this `[data 0, data 1, data2,.... data 143]`

                    dataset = dataset.reshape(-1)
                    print(dataset.shape) 



- The prediction problem is formulated in such a way that we want to look at the passenger number of Day 0 (today) to predict the pessenger number of Day 1 (tomorrow) i.e., there is a difference of 1 day between input feature and output label (`look back = 1`)   

- Take a look at the figure below to see how we convert a single sequence of data into supervised data points that contain input feature and a output label.  

<p align="center">
  <img src="https://user-images.githubusercontent.com/96804013/153277436-42e975b8-043a-4d8a-8752-eea10d93767e.png")
"/>

</p>

<p align="center">
  <em>Figure 2: Conversion of Sequential data to (x,y) format (Figure Drawn for look back = 1)</em>
</p>



Now, use the following code to perform this operation for the dataset:  

                    def gen_dataset(dset, look_back):
                      X = []
                      y = []
                      for i in range(len(dset)-look_back-1):
                        y_value = dset[i + look_back] # output label
                        x_value = dset[i : i + look_back] # input feature
                        y.append(y_value)
                        X.append(x_value)
                      
                      return np.array(X), np.array(y)


Now, make use of the function that you just wrote:

                    look_back = 1
                    data_X, data_y = gen_dataset(dataset, look_back)
                    print(data_X.shape, data_y.shape)

---
## Exercise 2
- Split the dataset (data_X, data_y) into `train` and `test` sets using `train_test_split` in the ration `70%` train and `30%` test. 
- Set `shuffle` to `True` and `random_state` to `42`
- Store the values in `X_train`, `X_test`, `y_train` and `y_test`.
- Print the shapes of `X_train`, `X_test`, `y_train` and `y_test`.

Reference ([link](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html))

---
## Exercise 3


Initialize a Sequential model with following code: 

                  lstm = Sequential()

Use `lstm.add()` function to build a LSTM model with the following architecture.

- 1st layer: LSTM layer with 100 units; specify `input_shape = (1, look_back)`
- 2nd layer (final layer): dense layer with one unit


Reference: LSTM ([link](https://keras.io/api/layers/recurrent_layers/lstm/)), Dense ([link](https://keras.io/api/layers/core_layers/dense/))


Now compile and train the model:

- Compile the model: set `loss` to `mean_squared_error` and `optimizer` to `adam`
- Train the model: set `epochs` to 10, `batch_size` to 1, and `verbose` to 2


Reference: Compile and fit ([link](https://keras.io/api/models/model_training_apis/))


---

- Its time to make predictions on both train and test set and see the error (RMSE) scores. Use the code below: 

- We have to perform the inverse-normalization operation to the predictions (i.e., convert the data back to the original scale). To do so, we make use of the same scaler that was instantiated and `fitted` earlier.

- Since we normalized the output label values as well we want to perform inverse transform operation on them as well. 

Use the following code: 

    
                    pred_train = lstm.predict(X_train)
                    pred_test = lstm.predict(X_test)

                    pred_train = scaler.inverse_transform(pred_train)
                    pred_test = scaler.inverse_transform(pred_test)

                    y_train = scaler.inverse_transform(y_train.reshape(-1,1))
                    y_test = scaler.inverse_transform(y_test.reshape(-1,1))
              
                    train_rmse = math.sqrt(mean_squared_error(y_train, pred_train))
                    test_rmse = math.sqrt(mean_squared_error(y_test, pred_test))

                    print(f"Train RMSE = {train_rmse} \nTest RMSE = {test_rmse}")

Next, let's plot a subset of the  model predictions as well as actual values on training set and test set using the following code. 
        
                    def plot(predictions, ground_truths, type = '', plot_x_size = 30):
                      predictions = predictions.reshape(-1)[0:plot_x_size]
                      ground_truths = ground_truths.reshape(-1)[0: plot_x_size]
                      x_axis = range(plot_x_size)

                      fig, ax = plt.subplots(figsize=(12,4), dpi = 80)
                      ax.plot(x_axis, predictions, alpha = 0.5) 
                      ax.plot(x_axis, predictions, 'o',label = "predictions") 
                      ax.plot(x_axis, ground_truths, alpha = 0.5) 
                      ax.plot(x_axis, ground_truths, 'x', label = "ground truths") 

                      ax.set_xticks(x_axis);
                      ax.set_xlabel("Day number")
                      ax.set_ylabel("Passenger Count")
                      ax.set_title(f"Model performance on a subset of {type} set")
                      ax.legend()
                      plt.show()

                    plot(pred_train, y_train, 'train')
                    plot(pred_test, y_test, 'test')

<p align="center">
  <img src="https://user-images.githubusercontent.com/96804013/153318480-c85d408d-41e0-4470-a815-ed1467e9b0dd.png")
"/>
</p>

<p align="center">
  <em>Figure 3:Visualization of LSTM predictions on train and test</em>
</p>
